# 1/ Cleaning data (converting types and removing NAs)

In [145]:
import pandas as pd, numpy as np
from tennis_predictor import clean_data

In [146]:
df = pd.read_csv("data.csv", low_memory=False)
COLUMNS = [
    "Location", "Tournament", "Date",
    "Series",
    "Court", "Surface",
    "Round", "Best of",
    "Winner", "Loser", "WRank", "LRank",
]
GAME_STATS_COLS = [
    "W1", "L1", "W2", "L2", "W3", "L3", "W4", "L4", "W5", "L5",
    "Wsets", "Lsets"
]
df = df[COLUMNS + GAME_STATS_COLS]

In [147]:
df.iloc[377]

Location                 Memphis
Tournament       Kroger St. Jude
Date                  14/02/2000
Series        International Gold
Court                     Indoor
Surface                     Hard
Round                  1st Round
Best of                        3
Winner                 Mamiit C.
Loser                  Jensen L.
WRank                        135
LRank                         NR
W1                             7
L1                             5
W2                             6
L2                             2
W3                           NaN
L3                           NaN
W4                           NaN
L4                           NaN
W5                           NaN
L5                           NaN
Wsets                          2
Lsets                          0
Name: 377, dtype: object

In [163]:
data = clean_data.convert_to_numeric(df, ["WRank", "LRank"])
data = data.assign(
    Date=pd.to_datetime(data.Date, format="%d/%m/%Y")
)
data = clean_data.drop_nas(data, COLUMNS)

Location       0
Tournament     0
Date           0
Series         0
Court          0
Surface        0
Round          0
Best of        0
Winner         0
Loser          0
WRank         16
LRank         98
dtype: int64


In [149]:
data.iloc[:,:20].head()

,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4
0,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,Ljubicic I.,63.0,77.0,6.0,4.0,6.0,2.0,NaN,NaN,NaN,NaN
1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Enqvist T.,Clement A.,5.0,56.0,6.0,3.0,6.0,3.0,NaN,NaN,NaN,NaN
2,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Escude N.,Baccanello P.,40.0,655.0,6.0,7.0,7.0,5.0,6.0,3.0,NaN,NaN
3,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Federer R.,Knippschild J.,65.0,87.0,6.0,1.0,6.0,4.0,NaN,NaN,NaN,NaN
4,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Fromberg R.,Woodbridge T.,81.0,198.0,7.0,6.0,5.0,7.0,6.0,4.0,NaN,NaN


In [150]:
data.iloc[:,20:].head()

,W5,L5,Wsets,Lsets
0,NaN,NaN,2.0,0.0
1,NaN,NaN,2.0,0.0
2,NaN,NaN,2.0,1.0
3,NaN,NaN,2.0,0.0
4,NaN,NaN,2.0,1.0


In [151]:
data.dtypes, data.shape

(Location              object
 Tournament            object
 Date          datetime64[ns]
 Series                object
 Court                 object
 Surface               object
 Round                 object
 Best of                int64
 Winner                object
 Loser                 object
 WRank                float64
 LRank                float64
 W1                   float64
 L1                   float64
 W2                   float64
 L2                   float64
 W3                   float64
 L3                   float64
 W4                   float64
 L4                   float64
 W5                   float64
 L5                   float64
 Wsets                float64
 Lsets                float64
 dtype: object, (46538, 24))

# 2/ Labelling players 1 and 2 (input: data Dataframe)

* The player specific columns are basically grouped into two groups: the various columns about the Winner and the various columns about the Loser
* We need to somehow pick, for each match a "Player1" and a "Player2", otherwise, otherwise all the observations would all belong to a single class! (ie the Winner always wins of course :P )

## Does the better ranked player always win?

In [152]:
matches_where_better_wins = data[data.WRank < data.LRank].shape[0]
matches_where_better_wins / data.shape[0]

0.6568395719626972

## They do about 65% of the time !!

#### So if we pick the better player as "Player1", we know that Player1 only wins 65% of the time. If we try to predict whether Player1 wins, we have 65% of "True" and "35%" of "False"

In [153]:
from tennis_predictor import transform_data
P1_P2 = transform_data.number_players(data, lambda x: x.WRank < x.LRank)
print(P1_P2.shape)
P1_P2.head()

(46538, 16)


,P1_Name,P1_Rank,P1_1,P1_2,P1_3,P1_4,P1_5,P1_Sets,P2_Name,P2_Rank,P2_1,P2_2,P2_3,P2_4,P2_5,P2_Sets
0,Dosedel S.,63.0,6.0,6.0,NaN,NaN,NaN,2.0,Ljubicic I.,77.0,4.0,2.0,NaN,NaN,NaN,0.0
1,Enqvist T.,5.0,6.0,6.0,NaN,NaN,NaN,2.0,Clement A.,56.0,3.0,3.0,NaN,NaN,NaN,0.0
2,Escude N.,40.0,6.0,7.0,6.0,NaN,NaN,2.0,Baccanello P.,655.0,7.0,5.0,3.0,NaN,NaN,1.0
3,Federer R.,65.0,6.0,6.0,NaN,NaN,NaN,2.0,Knippschild J.,87.0,1.0,4.0,NaN,NaN,NaN,0.0
4,Fromberg R.,81.0,7.0,5.0,6.0,NaN,NaN,2.0,Woodbridge T.,198.0,6.0,7.0,4.0,NaN,NaN,1.0


#### Let's now concatenate those new columns with the other columns that we care about (the first 8 columns in data, we don't the other Winner/Loser columns as we just copied/renamed them)

In [154]:
data_with_P1_P2 = data.iloc[:,:10].join(P1_P2, how="inner")
print(data_with_P1_P2.shape)
data_with_P1_P2.head()

(46538, 26)


,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,...,P1_5,P1_Sets,P2_Name,P2_Rank,P2_1,P2_2,P2_3,P2_4,P2_5,P2_Sets
0,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,Ljubicic I.,...,NaN,2.0,Ljubicic I.,77.0,4.0,2.0,NaN,NaN,NaN,0.0
1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Enqvist T.,Clement A.,...,NaN,2.0,Clement A.,56.0,3.0,3.0,NaN,NaN,NaN,0.0
2,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Escude N.,Baccanello P.,...,NaN,2.0,Baccanello P.,655.0,7.0,5.0,3.0,NaN,NaN,1.0
3,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Federer R.,Knippschild J.,...,NaN,2.0,Knippschild J.,87.0,1.0,4.0,NaN,NaN,NaN,0.0
4,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Fromberg R.,Woodbridge T.,...,NaN,2.0,Woodbridge T.,198.0,6.0,7.0,4.0,NaN,NaN,1.0


####  Let's add the target

In [155]:
data_with_P1_P2 = data_with_P1_P2.assign(
    Player1Wins=lambda x: x.P1_Name ==  x.Winner
)

In [156]:
if {"Winner", "Loser"}.intersection(data_with_P1_P2.columns):  # So that we can rerun this cell :)
    data_with_P1_P2 = data_with_P1_P2.drop(["Winner", "Loser"], axis=1)
print(data_with_P1_P2.columns)
data_with_P1_P2.Player1Wins.mean()

Index(['Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface', 'Round',
       'Best of', 'P1_Name', 'P1_Rank', 'P1_1', 'P1_2', 'P1_3', 'P1_4', 'P1_5',
       'P1_Sets', 'P2_Name', 'P2_Rank', 'P2_1', 'P2_2', 'P2_3', 'P2_4', 'P2_5',
       'P2_Sets', 'Player1Wins'],
      dtype='object')


0.65683957196269716

# 3/ Basic EDA (input: data_with_P1_P2)

At this point I haven't decided which features exactly I'm going to use:
* derived features (that would represent the players history up to the day of the match we'd try to predict), **excluding the ranks**
* the player's ranks augmented with the features above

In [157]:
eda_df = data_with_P1_P2.assign(RankDiff= lambda x: x.P2_Rank - x.P1_Rank)
corr_cols = [
    "Player1Wins", "P1_Rank", "P2_Rank", "RankDiff", "Best of"
]
eda_df[corr_cols].corr()

,Player1Wins,P1_Rank,P2_Rank,RankDiff,Best of
Player1Wins,1.000000,-0.120933,0.052750,0.088316,0.063309
P1_Rank,-0.120933,1.000000,0.356869,0.113219,-0.091759
P2_Rank,0.052750,0.356869,1.000000,0.968552,-0.027539
RankDiff,0.088316,0.113219,0.968552,1.000000,-0.004851
Best of,0.063309,-0.091759,-0.027539,-0.004851,1.000000


In [158]:
from sklearn.linear_model import LogisticRegression

In [159]:
lr = LogisticRegression()

X = eda_df[["P1_Rank", "P2_Rank", "RankDiff", "Best of"]]
Y = eda_df.Player1Wins
lr.fit(X, Y)
lr.score(X, Y)

0.65505608320082509

# 4/ Adding features derived from past history of the players (input: eda_df)

## For every point in time, we want to compute statistics about players **up until that point in time**

### Let's compute those statistics only starting from 2005, so that the players have enough history

### Games played

In [185]:
date_points = eda_df[eda_df.Date > "2005-01-01"].Date
date_points.head(), date_points.shape

(14518   2005-01-03
 14519   2005-01-03
 14520   2005-01-03
 14521   2005-01-03
 14522   2005-01-03
 Name: Date, dtype: datetime64[ns], (32057,))

In [201]:
all_players = pd.concat([eda_df.P1_Name, eda_df.P2_Name]).unique()
all_players.shape

(1355,)

In [209]:
def games_played(df, all_players):
    temp_df = df.copy()
    for p in all_players:
        temp_df[p + "_Played"] = (temp_df.P1_Name == p) | (temp_df.P2_Name == p)
        
    for p in all_players:
        temp_df[p + "_Won"] =  (
            (temp_df[p + "_Played"]) &  (
                ((temp_df.P1_Name == p) & (temp_df.Player1Wins)) |
                ((temp_df.P2_Name == p) & (~temp_df.Player1Wins))
            )
        )
         
    return temp_df

In [212]:
player_stats = games_played(eda_df, all_players)

In [215]:
player_stats["Federer R._Played"].sum(), player_stats["Federer R._Won"].sum()

(1234, 1016)

In [217]:

player_stats["Nadal R._Played"].sum(), player_stats["Nadal R._Won"].sum()

(950, 779)

In [225]:
my_df = pd.DataFrame({
    'A': np.arange(100),
    'B': np.linspace(-1, 1, 100),
})
my_df.head()

,A,B
0,0,-1.000000
1,1,-0.979798
2,2,-0.959596
3,3,-0.939394
4,4,-0.919192


In [236]:
my_df.rolling(12).apply(lambda x: sum(x))

,A,B
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN
